In [3]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import os
import shutil
print(os.listdir("../input"))

from glob import glob 
from skimage.io import imread
import gc

from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical

['labels.csv', 'train', 'test', 'sample_submission.csv']


Using TensorFlow backend.


In [4]:
base_tile_dir = '../input/train/'
df = pd.DataFrame({'path': glob(os.path.join(base_tile_dir,'*.jpg'))})
df['id'] = df.path.map(lambda x: x.split('/')[3].split(".")[0])
labels = pd.read_csv("../input/labels.csv")
df_data = df.merge(labels, on = "id")
df_data.head(3)

,path,id,breed
0,../input/train/dab2023fc149d0c42f905ffaa78b2fa...,dab2023fc149d0c42f905ffaa78b2fa6,african_hunting_dog
1,../input/train/b2e1a5fd7eedab2fad957eed10d6673...,b2e1a5fd7eedab2fad957eed10d66733,scotch_terrier
2,../input/train/bc7d13f323cfc48854ed90ca5207855...,bc7d13f323cfc48854ed90ca52078553,lakeland_terrier


In [5]:
# train_test_split # stratify=y creates a balanced validation set.
y = df_data['breed']
df_train, df_val = train_test_split(df_data, test_size=0.10, random_state=101, stratify=y)

In [6]:
# Create directories
train_path = 'base_dir/train'
valid_path = 'base_dir/valid'
test_path = '../input/test'
for fold in [train_path, valid_path]:
    for subf in y.unique():
        os.makedirs(os.path.join(fold, subf))

In [7]:
# Set the id as the index in df_data
df_data.set_index('id', inplace=True)
df_data.head()

,path,breed
id,,
dab2023fc149d0c42f905ffaa78b2fa6,../input/train/dab2023fc149d0c42f905ffaa78b2fa...,african_hunting_dog
b2e1a5fd7eedab2fad957eed10d66733,../input/train/b2e1a5fd7eedab2fad957eed10d6673...,scotch_terrier
bc7d13f323cfc48854ed90ca52078553,../input/train/bc7d13f323cfc48854ed90ca5207855...,lakeland_terrier
f644ec58aab5f57925bd55969f4b925b,../input/train/f644ec58aab5f57925bd55969f4b925...,kelpie
9cc4a7f129eca8f31387c624ada69790,../input/train/9cc4a7f129eca8f31387c624ada6979...,lhasa


In [8]:
for image in df_train['id'].values:
    fname = image + '.jpg'
    breed = str(df_data.loc[image,'breed']) # get the label for a certain image
    src = os.path.join('../input/train', fname)
    dst = os.path.join(train_path, breed, fname)
    shutil.copyfile(src, dst)
    
for image in df_val['id'].values:
    fname = image + '.jpg'
    breed = str(df_data.loc[image,'breed']) # get the label for a certain image
    src = os.path.join('../input/train', fname)
    dst = os.path.join(valid_path, breed, fname)
    shutil.copyfile(src, dst)

In [10]:

print(os.listdir("base_dir/valid/groenendael"))
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

#img=mpimg.imread('base_dir/valid/groenendael/9e90e30a460f9c139984766dd957e62a.jpg')
#plt.imshow(img)

['6fb144599ec91a40edeac0d884ff43b2.jpg', 'db3e4f1de38bee86c0c1aba6f847d6db.jpg', '782319adbba9e5002b9f727abb845543.jpg', 'a745bd8479c64408093c2116a58e38f6.jpg', '38c5b9c06dd0345604cb1a818ccc37e3.jpg', '298484d02269df72a0776e5a0bf76973.jpg', '28f7140259e0f9d36cd67cb3f194fe34.jpg', '3a1eeae505c9fedcfd97d3eaa94c3c66.jpg']


In [11]:
from keras.preprocessing.image import ImageDataGenerator

IMAGE_SIZE = 200
num_train_samples = len(df_train)
num_val_samples = len(df_val)
train_batch_size = 32
val_batch_size = 32

train_steps = np.ceil(num_train_samples / train_batch_size)
val_steps = np.ceil(num_val_samples / val_batch_size)

datagen = ImageDataGenerator(
    rescale=1./255,
    # preprocessing_function=lambda x:(x - x.mean()) / x.std() if x.std() > 0 else x,
    horizontal_flip=True,
    vertical_flip=True)

train_gen = datagen.flow_from_directory(train_path,
                                        target_size=(IMAGE_SIZE,IMAGE_SIZE),
                                        batch_size=train_batch_size,
                                        class_mode='categorical')

val_gen = datagen.flow_from_directory(valid_path,
                                        target_size=(IMAGE_SIZE,IMAGE_SIZE),
                                        batch_size=val_batch_size,
                                        class_mode='categorical')

# Note: shuffle=False causes the test dataset to not be shuffled
test_gen = datagen.flow_from_directory(valid_path,
                                        target_size=(IMAGE_SIZE,IMAGE_SIZE),
                                        batch_size=1,
                                        class_mode='categorical',
                                        shuffle=False)

Found 9199 images belonging to 120 classes.
Found 1023 images belonging to 120 classes.
Found 1023 images belonging to 120 classes.


In [12]:
"""
from keras.applications.inception_resnet_v2 import InceptionResNetV2
from keras.models import Sequential, Model
from keras import layers
from keras import optimizers
from keras.layers import Concatenate, BatchNormalization, Flatten, Dense, Input, Dropout, MaxPooling2D, Concatenate, GlobalMaxPooling2D, GlobalAveragePooling2D, Lambda, Multiply, LSTM, Bidirectional, PReLU, MaxPooling1D

IMG_SIZE = (200, 200)
IN_SHAPE = (*IMG_SIZE, 3)

dropout_dense=0.5

conv_base = InceptionResNetV2(
    weights='imagenet',
    include_top=False,
    input_shape=IN_SHAPE
)

model = Sequential()
model.add(conv_base)
model.add(layers.Flatten())
model.add(layers.Dense(256, use_bias=False))
model.add(layers.BatchNormalization())
model.add(layers.Activation("relu"))
model.add(layers.Dropout(dropout_dense))
model.add(layers.Dense(120, activation = "sigmoid"))

conv_base.summary()

conv_base.Trainable=True

from keras import optimizers

# conv_base.trainable = False
model.compile(optimizers.Adam(0.0001), loss = 'categorical_crossentropy', metrics=["accuracy"])

from keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
h5_path = "inception_resnet_v2_model.h5"
checkpoint = ModelCheckpoint(h5_path, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
earlystopper = EarlyStopping(monitor='val_loss', patience=2, verbose=1, restore_best_weights=True)
reducel = ReduceLROnPlateau(monitor='val_loss', patience=2, verbose=1, factor=0.5, min_lr = 0.000000001)
history = model.fit_generator(train_gen, steps_per_epoch=train_steps, 
                    validation_data=val_gen,
                    validation_steps=val_steps,
                    epochs=20,
                   callbacks=[reducel, checkpoint])
"""

In [14]:
from keras.applications.inception_resnet_v2 import InceptionResNetV2
from keras.models import Sequential, Model
from keras import layers
from keras import optimizers
from keras.layers import Concatenate, BatchNormalization, Flatten, Dense, Input, Dropout, MaxPooling2D, Concatenate, GlobalMaxPooling2D, GlobalAveragePooling2D, Lambda, Multiply, LSTM, Bidirectional, PReLU, MaxPooling1D

IMG_SIZE = (200, 200)
IN_SHAPE = (*IMG_SIZE, 3)

dropout_dense=0.5

conv_base = InceptionResNetV2(
    weights='imagenet',
    include_top=False,
    input_shape=IN_SHAPE
)

model = Sequential()
model.add(conv_base)
model.add(GlobalAveragePooling2D())
model.add(layers.Dense(120, activation = "sigmoid"))

conv_base.summary()

conv_base.Trainable=True

from keras import optimizers

# conv_base.trainable = False
model.compile(optimizers.Adam(0.0001), loss = 'categorical_crossentropy', metrics=["accuracy"])

Instructions for updating:
Colocations handled automatically by placer.
219062272/219055592 [==============================] - 3s 0us/step
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 200, 200, 3)  0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 99, 99, 32)   864         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 99, 99, 32)   96          conv2d_1[0][0]                   
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 99, 99, 32)   0           batc

                                                                 block8_2_conv[0][0]              
__________________________________________________________________________________________________
block8_2_ac (Activation)        (None, 4, 4, 2080)   0           block8_2[0][0]                   
__________________________________________________________________________________________________
conv2d_173 (Conv2D)             (None, 4, 4, 192)    399360      block8_2_ac[0][0]                
__________________________________________________________________________________________________
batch_normalization_173 (BatchN (None, 4, 4, 192)    576         conv2d_173[0][0]                 
__________________________________________________________________________________________________
activation_173 (Activation)     (None, 4, 4, 192)    0           batch_normalization_173[0][0]    
__________________________________________________________________________________________________
conv2d_174

In [15]:
from keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
h5_path = "inception_resnet_v2_model1.h5"
checkpoint = ModelCheckpoint(h5_path, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
earlystopper = EarlyStopping(monitor='val_loss', patience=2, verbose=1, restore_best_weights=True)
reducel = ReduceLROnPlateau(monitor='val_loss', patience=2, verbose=1, factor=0.5, min_lr = 0.000000001)
history = model.fit_generator(train_gen, steps_per_epoch=train_steps, 
                    validation_data=val_gen,
                    validation_steps=val_steps,
                    epochs=20,
                   callbacks=[reducel, checkpoint])

Instructions for updating:
Use tf.cast instead.
Epoch 1/20
288/288 [==============================] - 182s 631ms/step - loss: 3.0577 - acc: 0.3868 - val_loss: 1.8794 - val_acc: 0.5337

Epoch 00001: val_acc improved from -inf to 0.53372, saving model to inception_resnet_v2_model1.h5
Epoch 2/20
288/288 [==============================] - 121s 422ms/step - loss: 1.4048 - acc: 0.6241 - val_loss: 1.4534 - val_acc: 0.6002

Epoch 00002: val_acc improved from 0.53372 to 0.60020, saving model to inception_resnet_v2_model1.h5
Epoch 3/20
288/288 [==============================] - 121s 418ms/step - loss: 0.9220 - acc: 0.7401 - val_loss: 4.2539 - val_acc: 0.1144

Epoch 00003: val_acc did not improve from 0.60020
Epoch 4/20
288/288 [==============================] - 120s 418ms/step - loss: 0.6570 - acc: 0.8094 - val_loss: 5.0511 - val_acc: 0.0196

Epoch 00004: ReduceLROnPlateau reducing learning rate to 4.999999873689376e-05.

Epoch 00004: val_acc did not improve from 0.60020
Epoch 5/20
288/288 [====